In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ParsMap.xlsx


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"","key":""}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle kernels output aylinnaebzadeh/persianmind-v333 -p /kaggle/working

Output file downloaded to /kaggle/working/PersianMind-V1.0-formalizer-lora/README.md
Output file downloaded to /kaggle/working/PersianMind-V1.0-formalizer-lora/adapter_config.json
Output file downloaded to /kaggle/working/PersianMind-V1.0-formalizer-lora/adapter_model.safetensors
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/README.md
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/adapter_config.json
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/adapter_model.safetensors
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/added_tokens.json
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/optimizer.pt
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/rng_state.pth
Output file downloaded to /kaggle/working/persianmind-formalizer-lora/checkpoint-2500/scaler.pt
Output

# Sources
- https://tuanatran.medium.com/fine-tuning-large-language-model-with-hugging-face-pytorch-adce80dce2ad
- https://www.kaggle.com/code/shayanbemanian/persian-informal-to-formal-transformation

# W&B first of all:)

In [7]:
import warnings
warnings.filterwarnings('ignore') 
# ------------------------
!pip uninstall -y --quiet wandb
!pip install --quiet wandb==0.17.5 
# ------------------------
import wandb
!wandb offline
!wandb disabled
# ------------------------
import os
os.environ["WANDB_DISABLED"] = "true"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.9 MB/s eta 0:00:0000:0100:01
W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
W&B disabled.


# Install libraries

In [8]:
!pip install --quiet --upgrade datasets transformers evaluate sentencepiece accelerate 
!pip install --quiet dadmatools[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 78.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:0

In [9]:
!pip install -U --quiet peft accelerate
!pip install -U --quiet bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 10.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 27.9 MB/s eta 0:00:00:00:0100:01


In [10]:
!pip install -U --quiet rouge-score

  Preparing metadata (setup.py) ... done


# Libraries

In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from dadmatools.normalizer import Normalizer

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, random_split
from torch.optim import AdamW

from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, get_linear_schedule_with_warmup
from transformers import  BitsAndBytesConfig, DataCollatorForLanguageModeling, TrainingArguments, Trainer

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel


import seaborn as sns
import matplotlib.pyplot as plt

import re
import random
import time
import datetime
import os

2025-08-17 14:47:23.073298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755442043.239053      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755442043.285453      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
tqdm.pandas()

# Read input data

In [13]:
df_parsmap = pd.read_excel("/kaggle/input/ParsMap.xlsx")
df_parsmap.head()

,inFormalForm,formalForm,formalWords,inFormalWords
0,من دوس دارم برم خونه درس بخونم.,من دوست دارم که به خانه بروم تا درس بخوانم.,من/دوست/دارم/بروم/خانه/درس/بخوانم,من/دوس/دارم/برم/خونه/درس/بخونم
1,میتونی منو ببری خونمون یکم نون وردارم؟,می‌توانی من را به خانه‌مان ببری تا کمی نان برد...,می‌توانی/من را/ببری/خانه‌مان/کمی/نان/بردارم,میتونی/منو/ببری/خونمون/یکم/نون/وردارم
2,رفتم درمونگاه واسه بچم دوا بگیرم، یهو بارون گرفت,به درمانگاه رفتم تا برای بچه‌ام دوا بگیرم که ی...,رفتم/درمانگاه/برای/بچه‌ام/دوا/بگیرم/یکهو/باران...,رفتم/درمونگاه/واسه/بچم/دوا/بگیرم/یهو/بارون/گرفت
3,اون که خدا شاهده مث دیوونه هاس,خدا شاهد است که او مثل دیوانه‌ها است.,شاهد است/او/که/خدا/مثل/دیوانه‌ها است,شاهده/اون/که/خدا/مث/دیوونه هاس
4,یه سری آدما هستن که هر قدر خوبی کنی اصلن انگار...,یک سری آدم‌ها هستند که هر قدر هم خوبی کنی، اصل...,یک/سری/آدم‌ها/هستند/که/هر/قدر/خوبی/کنی/اصلاً/ا...,یه/سری/آدما/هستن/که/هر/قدر/خوبی/کنی/اصلن/انگار...


In [14]:
normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=True,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None
)

text = 'من دوس دارم برم خونه درس بخونم    .'
print(normalizer.normalize(text))

من دوس دارم برم خونه درس بخونم.


In [15]:
def normalize_text(text):
    if pd.isna(text):
        return text
    return normalizer.normalize(str(text))

In [16]:
df_parsmap['formalForm'] = df_parsmap['formalForm'].progress_apply(normalize_text)
df_parsmap['inFormalForm'] = df_parsmap['inFormalForm'].progress_apply(normalize_text)

df_parsmap.head()

100%|██████████| 50014/50014 [00:06<00:00, 7714.56it/s]


,inFormalForm,formalForm,formalWords,inFormalWords
0,من دوس دارم برم خونه درس بخونم.,من دوست دارم که به خانه بروم تا درس بخوانم.,من/دوست/دارم/بروم/خانه/درس/بخوانم,من/دوس/دارم/برم/خونه/درس/بخونم
1,میتونی منو ببری خونمون یکم نون وردارم؟,می‌توانی من را به خانه‌مان ببری تا کمی نان برد...,می‌توانی/من را/ببری/خانه‌مان/کمی/نان/بردارم,میتونی/منو/ببری/خونمون/یکم/نون/وردارم
2,رفتم درمونگاه واسه بچم دوا بگیرم، یهو بارون گرفت,به درمانگاه رفتم تا برای بچه‌ام دوا بگیرم که ی...,رفتم/درمانگاه/برای/بچه‌ام/دوا/بگیرم/یکهو/باران...,رفتم/درمونگاه/واسه/بچم/دوا/بگیرم/یهو/بارون/گرفت
3,اون که خدا شاهده مث دیوونه هاس,خدا شاهد است که او مثل دیوانه‌ها است.,شاهد است/او/که/خدا/مثل/دیوانه‌ها است,شاهده/اون/که/خدا/مث/دیوونه هاس
4,یه سری آدما هستن که هر قدر خوبی کنی اصلن انگار...,یک سری آدم‌ها هستند که هر قدر هم خوبی کنی، اصل...,یک/سری/آدم‌ها/هستند/که/هر/قدر/خوبی/کنی/اصلاً/ا...,یه/سری/آدما/هستن/که/هر/قدر/خوبی/کنی/اصلن/انگار...


In [17]:
df_parsmap = df_parsmap.dropna(subset=["inFormalForm", "formalForm"])

In [18]:
base_model_id = "universitytehran/PersianMind-v1.0"
compute_dtype = torch.bfloat16 if torch.cuda.get_device_capability(0)[0] >= 8 else torch.float16

print("Compute dtype:", compute_dtype)

Compute dtype: torch.float16


In [19]:
def safe_str(x):
    return "" if x is None or (isinstance(x, float) and np.isnan(x)) else str(x)

df = df_parsmap.copy()
df = df.dropna(subset=["inFormalForm","formalForm"])  # keep only rows with both sides

def make_text(row):
    informal = safe_str(row["inFormalForm"])
    formal   = safe_str(row["formalForm"])
    return f"<s><|startoftext|>[Informal]{informal}[Formal]{formal}<|endoftext|>"

df["text"] = df.apply(make_text, axis=1)

In [20]:
# 90/10 split
perm = np.random.permutation(len(df))
cut = int(0.9*len(df))
train_df = df.iloc[perm[:cut]].reset_index(drop=True)
val_df   = df.iloc[perm[cut:]].reset_index(drop=True)

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df[["text"]]),
    "validation": Dataset.from_pandas(val_df[["text"]]),
})
len(ds["train"]), len(ds["validation"])

(45011, 5002)

In [16]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True, trust_remote_code=True)

specials = {
    "bos_token": "<s>",
    "eos_token": "</s>",
    "pad_token": "<pad>",
}
# Make sure tokens exist
for k,v in specials.items():
    if getattr(tokenizer, k, None) != v:
        tokenizer.add_special_tokens({k: v})

added = tokenizer.add_tokens(["<|startoftext|>", "<|endoftext|>", "[Informal]", "[Formal]", "<sep>"], special_tokens=True)
print("Added new tokens:", added)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/688k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

Added new tokens: 5


In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
)

model.resize_token_embeddings(len(tokenizer))

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False  

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [18]:

lora_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(model, lora_config)

model.gradient_checkpointing_enable()

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,} / Total: {total:,} ({100*trainable/total:.2f}%)")


Trainable: 39,976,960 / Total: 3,618,344,960 (1.10%)


In [19]:
max_length = 64

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )

tokenized = ds.map(tokenize_batch, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/45011 [00:00<?, ? examples/s]

Map:   0%|          | 0/5002 [00:00<?, ? examples/s]

In [20]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [21]:
effective_bs = 16 
per_device_train_bs = 2
per_device_eval_bs = 2
grad_accum = max(1, effective_bs // per_device_train_bs)
epochs = 1

args = TrainingArguments(
    output_dir="./persianmind-formalizer-lora",
    num_train_epochs=epochs,
    per_device_train_batch_size=per_device_train_bs,
    per_device_eval_batch_size=per_device_eval_bs,
    gradient_accumulation_steps=grad_accum,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    weight_decay=0.001,
    logging_steps=50,

    eval_strategy="steps",
    eval_steps=500,             
    save_total_limit=2,
    load_best_model_at_end=True,

    bf16=(compute_dtype==torch.bfloat16),
    fp16=(compute_dtype==torch.float16),

    optim="paged_adamw_8bit",          

    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    dataloader_persistent_workers=True,

    group_by_length=True,              
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=collator,
    tokenizer=tokenizer,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss,Validation Loss
500,3.120800,3.174871
1000,2.977600,3.109787
1500,2.922500,3.031685


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

For saving the model and pushing it to huggingface:

- https://www.kaggle.com/code/sureshbeekhani/fine-tune-llama-2-7b-ipynb#Step-7:-Store-New-Llama2-Model-(Llama-2-7b-chat-finetune)

In [ ]:
new_model = "PersianMind-V1.0-formalizer-lora"

In [ ]:
trainer.model.save_pretrained(new_model)

In [21]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/persianmind-formalizer-lora/checkpoint-2814")

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

base_model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(base_model, "/kaggle/working/persianmind-formalizer-lora/checkpoint-2814")

model = model.merge_and_unload()

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from huggingface_hub import login

api_token = ""  
login(token=api_token)

model_name = "AylinNaebzadeh/PersianMind-V1.0-formalizer-qlora"

model.push_to_hub(model_name, check_pr=True)

tokenizer.push_to_hub(model_name, check_pr=True)

In [28]:
import dadmatools.pipeline.language as language

# here lemmatizer and pos tagger will be loaded
# as tokenizer is the default tool, it will be loaded as well even without calling
pips = 'tok'
nlp = language.Pipeline(pips)
# doc is now a proper spaCy Doc object
doc = nlp('کشور بزرگ ایران توانسته در طی سال‌ها اقشار مختلفی از قومیت‌های گوناگون را به خوبی در خودش جا بده')

Loading pretrained XLM-Roberta, this may take a while...
Model fa_tokenizer exists in cache/dadmatools/fa_tokenizer.pt
Loading tokenizer for persian
Loading multi-word expander for persian
Active language: persian


In [29]:
doc

کشور بزرگ ایران توانسته در طی سال‌ها اقشار مختلفی از قومیت‌های گوناگون را به خوبی در خودش جا بده

In [40]:
import torch, re, random
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize

def generator(informal, max_length=64, num_return_sequences=1):
    # <s><|startoftext|>[Informal]{informal}[Formal]{formal}<|endoftext|>
    model.eval()
    prompt = f"<s><|startoftext|>[Informal]{normalize_text(informal)}[Formal]"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    decoded_outputs = model.generate(
        **inputs,
        do_sample=True,
        top_k=50,
        max_new_tokens=64,
        top_p=0.95,
        num_return_sequences=num_return_sequences
    )

    outputs = []
    for output in decoded_outputs:
        o = tokenizer.decode(output, skip_special_tokens=False)
        o = o.replace("<s>", "").replace("</s>", "").replace("<|startoftext|>", "").replace("<|endoftext|>", "").replace("<pad>", "")
        pattern = r'\[Informal\].*?\[Formal\]'
        o = re.sub(pattern, '', o)
        outputs.append(o.strip())

    return outputs

In [39]:
from nltk.tokenize import word_tokenize

text = doc.text
words = word_tokenize("سلام چطوری")
words

['سلام', 'چطوری']

In [41]:
sample_size = 1000
sample_indices = random.sample(range(len(df_parsmap)), sample_size)

reference_texts = [df_parsmap.iloc[i]['formalForm'] for i in sample_indices]
# generated_texts = [generator(df_parsmap.iloc[i]['inFormalForm'], num_return_sequences=1)[0] for i in sample_indices]

informal_texts = [df_parsmap.iloc[i]['inFormalForm'] for i in sample_indices]

generated_texts = []
for text in tqdm(informal_texts, desc="Generating outputs"):
    generated_texts.append(generator(text, num_return_sequences=1)[0])

reference_tokens = [word_tokenize(text) for text in reference_texts]
generated_tokens = [word_tokenize(text) for text in generated_texts]

bleu_scores = []
smoothing_func = SmoothingFunction().method1

for ref_tokens, gen_tokens in zip(reference_tokens, generated_tokens):
    bleu = sentence_bleu([ref_tokens], gen_tokens, smoothing_function=smoothing_func)
    bleu_scores.append(bleu)

avg_bleu_score = sum(bleu_scores) / len(bleu_scores)

print(f"Average BLEU score on {sample_size} samples: {avg_bleu_score:.4f}")


Generating outputs: 100%|██████████| 1000/1000 [1:03:21<00:00,  3.80s/it]


Average BLEU score on 1000 samples: 0.2516
